In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
end

using Random
using Distributions
using Plots
using StatsPlots
using prml
using prml: pdf

Random.seed!(1234);
rng = MersenneTwister(1234);
gr();

In [ ]:
function create_toy_data(func, n_samples::Int64, std=1.0, domain=[0.0, 1.0])
    x = collect(range(domain[1], domain[2], length=n_samples));
    shuffle!(rng, x);
    noise = rand(Uniform(-std, std), n_samples);
    return x, func.(x) + noise
end

function sinusoidal(x)
    return sin.(2 * pi .* x)
end

## 6.1 Dual Representation

In [ ]:
x_train, y_train = create_toy_data(sinusoidal, 10, 0.2);

x_train = reshape(x_train, 1, size(x_train)[1])
model = GaussianProcessRegressor(PolynomialKernel(3, 1.0), 1e10)
fitting(model, x_train, y_train);

x = reshape(collect(range(0.0, 1.0, length=100)), 1, 100);
y, y_sigmas = predict(model, x, true)

plot(x[1, :], y[1, :], lw=2, color="green", label="regression")
plot!(x_train[1, :], y_train, seriestype="scatter", label="training")
plot!(x[1, :], sin.(2 * pi .* x[1, :]), lw=2, color="red", label="true")

### 6.3.1 Nadarya-Watson model

Noise added to **input**

In [ ]:
function create_toy_data2(func, n_samples::Int64, sigma, domain)
    x = collect(range(domain[1], domain[2], length=n_samples));
    shuffle!(rng, x);
    noise = rand(Normal(0, sigma), n_samples);
    return x, func(x + noise)
end
function g(mu::Float64, sigma2::Float64)
    return exp(-0.5 * mu^2 / sigma2) / sqrt(2.0 * pi * sigma2)
end

N = 10
sigma = 0.1
sigma2 = sigma^2
x_train, y_train = create_toy_data2(sinusoidal, N, sigma, [0.0, 1.0]);
x_start = min(0.0, minimum(x_train))
x_end = max(1.0, maximum(x_train))
x = collect(range(x_start, x_end, length=100))
y = zeros(100)
y_vars = zeros(100)
for i in 1:100
    gs = [g(x[i] - x_train[n], sigma2) for n in 1:N]
    hs = zeros(N)
    for n in 1:N
        h_n = gs[n] / sum(gs)
        hs[n] = h_n
        y[i] += y_train[n] * h_n
    end
    y_vars[i] = sum([ hs[n] * (y_train[n]^2 + sigma2) for n in 1:N]) - (sum([ hs[n] * y_train[n] for n in 1:N]))^2
end

y_sigma = sqrt.(y_vars)
plot(x, y, ribbon=(y_sigma * 2.0, y_sigma * 2.0), label="regression")
plot!(x_train, y_train, seriestype="scatter", label="training")
plot!(x, sinusoidal(x), lw=2, label="sin")

<img src="images/ch6/image1.png">

## 6.4 Gaussian Process

### 6.4.2 Gaussian process for regression

In [ ]:
x_train, y_train = create_toy_data(sinusoidal, 7, 0.1, [0.0, 0.7]);
x_train = reshape(x_train, 1, size(x_train)[1])
model = GaussianProcessRegressor(RBFKernel([1.0, 15.0]), 100);
fitting(model, x_train, y_train)

x = reshape(collect(range(0.0, 1.0, length=100)), 1, 100);
y, y_sigmas = predict(model, x, true)

y_std = sqrt.(reshape(y_sigmas, length(y_sigmas)))
plot(x[1, :], y[1, :], ribbon=(y_std, y_std), color="green", label="std")
plot!(x_train[1, :], y_train, seriestype="scatter", label="training")
plot!(x[1, :], sin.(2 * pi .* x[1, :]), lw=2, color="red", label="true")

<img src="images/ch6/image2.png">

### 6.4.3 Learning the hyperparameters

In [ ]:
plots = []
x_train, y_train = create_toy_data(sinusoidal, 7, 0.1, [0.0, 0.7]);
x_train = reshape(x_train, 1, size(x_train)[1])
rbf_kernel = RBFKernel([1.0, 1.0])
model = GaussianProcessRegressor(rbf_kernel, 100);
fitting(model, x_train, y_train)

x = reshape(collect(range(0.0, 1.0, length=100)), 1, 100);
y, y_sigmas = predict(model, x, true)

y_std = sqrt.(reshape(y_sigmas, length(y_sigmas)))
gpr_params = map(x -> round(x, sigdigits=3), rbf_kernel._params)
p = plot(x[1, :], y[1, :], ribbon=(y_std, y_std), color="green", label="std", title="gpr = $(gpr_params)")
p = plot!(x_train[1, :], y_train, seriestype="scatter", label="training")
p = plot!(x[1, :], sin.(2 * pi .* x[1, :]), lw=2, color="red", label="true")
push!(plots, p)

x_train, y_train = create_toy_data(sinusoidal, 7, 0.1, [0.0, 0.7]);
x_train = reshape(x_train, 1, size(x_train)[1])
model = GaussianProcessRegressor(rbf_kernel, 100);
fitting(model, x_train, y_train, 100)

x = reshape(collect(range(0.0, 1.0, length=100)), 1, 100);
y, y_sigmas = predict(model, x, true)

y_std = sqrt.(reshape(y_sigmas, length(y_sigmas)))
gpr_params = map(x -> round(x, sigdigits=3), rbf_kernel._params)
p = plot(x[1, :], y[1, :], ribbon=(y_std, y_std), color="green", label="std", title="gpr = $(gpr_params)")
p = plot!(x_train[1, :], y_train, seriestype="scatter", label="training")
p = plot!(x[1, :], sin.(2 * pi .* x[1, :]), lw=2, color="red", label="true")
push!(plots, p)

plot(plots[1], plots[2], size=(600, 300))

<img src="images/ch6/image3.png">

### 6.4.4 Automatic relevance determination

In [ ]:
function create_toy_data_3d(func, n_samples, std)
    x0 = range(0.0, 1.0, length=n_samples)
    x1 = x0 + rand(Normal(0, std), n_samples)
    x2 = rand(Normal(0, std), n_samples)
    t = func(x0) + rand(Normal(0, std), n_samples)
    return vcat(transpose(x0), transpose(x1), transpose(x2)), t
end

x_train, y_train = create_toy_data_3d(sinusoidal, 20, 0.1)
rbf_kernel = RBFKernel([1.0, 1.0, 1.0, 1.0])
model = GaussianProcessRegressor(rbf_kernel, 100)

max_iter = 0
learning_rate = 0.01
fitting(model, x_train, y_train, max_iter, learning_rate)

plots = []

x0 = range(0.0, 1.0, length=100)
x1 = x0 + rand(Normal(0, 0.1), 100)
x2 = rand(Normal(0, 0.1), 100)
x = vcat(transpose(x0), transpose(x1), transpose(x2))
y, y_std = predict(model, x, true)
y_vars = sqrt.(reshape(y_std, length(y_std)))
p = plot(x_train[1, :], y_train, seriestype="scatter", label="train")
p = plot!(x[1, :], sinusoidal(x[1, :]), label="sin(2 pi x)")
p = plot!(x[1, :], y[1, :], ribbon=(y_vars, y_vars), label="gpr")
push!(plots, p)

max_iter = 100
learning_rate = 0.01
fitting(model, x_train, y_train, max_iter, learning_rate)

x0 = range(0.0, 1.0, length=100)
x1 = x0 + rand(Normal(0, 0.1), 100)
x2 = rand(Normal(0, 0.1), 100)
x = vcat(transpose(x0), transpose(x1), transpose(x2))
y, y_std = predict(model, x, true)
y_vars = sqrt.(reshape(y_std, length(y_std)))
p = plot(x_train[1, :], y_train, seriestype="scatter", label="train")
p = plot!(x[1, :], sinusoidal(x[1, :]), label="sin(2 pi x)")
p = plot!(x[1, :], y[1, :], ribbon=(y_vars, y_vars), label="gpr")
push!(plots, p)

plot(plots[1], plots[2], size=(600, 300))

<img src="images/ch6/image4.png">